In [2]:
import numpy as np
import pandas as pd

### 1. Taking out unique cell tower locations and saving them into a new csv file

In [2]:
userDistr_data = pd.read_csv("user_distribution_new.csv")

userDistr_data.columns.values

array(['location(latitude/lontitude)', 'start date', 'time', 'No.of user'],
      dtype=object)

In [11]:
userDistr_data.head(100)

,location(latitude/lontitude),start date,time,No.of user
0,22.522803/114.218796,2014-05-31,23,1
1,22.522803/114.218796,2014-06-01,0,1
2,22.522803/114.218796,2014-06-01,1,1
3,22.522803/114.218796,2014-06-01,2,2
4,22.522803/114.218796,2014-06-01,3,1
...,...,...,...,...
95,22.522803/114.218796,2014-06-05,13,5
96,22.522803/114.218796,2014-06-05,14,4
97,22.522803/114.218796,2014-06-05,15,3
98,22.522803/114.218796,2014-06-05,16,3


In [23]:
## split the "location(latitude/lontitude)" column and separate latitude & longitude into separate columns into a new dataframe
location_coordinates = userDistr_data['location(latitude/lontitude)'].str.split("/", n=1, expand=True)

location_coordinates.head(2)

,0,1
0,22.522803,114.218796
1,22.522803,114.218796


In [24]:
type(location_coordinates)

pandas.core.frame.DataFrame

In [27]:
d = {"latitude": location_coordinates[0], "longitude": location_coordinates[1]}

## create a new dataFrame with only latitude & longitude
cell_tower_locations = pd.DataFrame(data=d)
cell_tower_locations.head()

,latitude,longitude
0,22.522803,114.218796
1,22.522803,114.218796
2,22.522803,114.218796
3,22.522803,114.218796
4,22.522803,114.218796


In [26]:
cell_tower_locations.shape

(346820, 2)

In [31]:
## drop duplicate locations
cell_tower_locations = cell_tower_locations.drop_duplicates()

In [32]:
cell_tower_locations.shape

(2769, 2)

In [33]:
cell_tower_locations.head(10)

,latitude,longitude
0,22.522803,114.218796
313,24.284812,102.999068
461,25.222206,117.086322
494,26.139329,103.078562
845,26.215115,109.744661
1009,28.738742,120.640606
1073,28.812629,115.952954
1076,29.151779,120.985563
1080,29.263844,115.023159
1310,29.526266,119.910488


In [50]:
print(cell_tower_locations.columns.dtype)

cell_tower_locations = cell_tower_locations.astype({"latitude": float, "longitude": float})

object


In [45]:
## change the dtypes of latitude & longitude columns into float64
cell_tower_locations.dtypes

latitude     float64
longitude    float64
dtype: object

In [46]:
## save the unique cell tower locations into a new .csv file
cell_tower_locations.to_csv("cell_tower_locations.csv", index=False)

tower_locations is a Pandas DataFrame. It contains the location of cell towers as latitude, longitude

In [47]:
tower_locations = pd.read_csv("cell_tower_locations.csv")

tower_locations.head(10)

,latitude,longitude
0,22.522802,114.218796
1,24.284813,102.999069
2,25.222206,117.086319
3,26.139330,103.078560
4,26.215115,109.744659
5,28.738743,120.640610
6,28.812630,115.952957
7,29.151779,120.985565
8,29.263844,115.023155
9,29.526266,119.910484


In [48]:
tower_locations.shape

(2769, 2)

### 2. For a vehicle in Vehicular Dataset, calculate the closest cell towers. 
Distance Metric : Haversine Distance

tower_locations is a pandas dataFrame, which contains coordinates(lat,long) of cell towers

In [3]:
## cell tower location dataset
tower_locations = pd.read_csv("cell_tower_locations.csv")

tower_locations.shape

(2769, 2)

In [22]:
tower_locations.head(2)

,latitude,longitude
0,22.522802,114.218796
1,24.284813,102.999069


data_0 is a pandas DataFrame. It contains the log of diffrent vehicles. It is one .txt file out of 2500 such files in VEHICULAR DATASET

In [4]:
data_0 = pd.read_csv("1804010000.txt", sep="|", header=None)

data_0.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,18852,A,0,0,0,1,0,0,2018-04-01,2018-03-31 23:59:48,121.498510,31.289317,68.0,94.0,8,069
1,27772,A,0,1,1,0,0,0,2018-04-01,2018-03-31 23:59:59,121.560292,31.244518,47.0,223.0,9,047
2,10373,A,0,1,5,0,0,0,2018-04-01,2018-04-01 00:00:02,121.412063,31.278722,0.0,203.0,8,000
3,10708,A,0,1,1,0,0,0,2018-04-01,2018-04-01 00:00:02,121.666038,31.165347,0.0,339.0,9,000
4,17988,A,0,0,0,0,0,0,2018-04-01,2018-04-01 00:00:01,121.460905,31.342570,34.0,173.0,9,030


Field description in VEHICULAR DATASET:

 

0 : Car number

1 : Control word (A: normal, M: alarm)

2 : Business status (0: normal, 1: alarm)

3 : Passenger carrying status (0: heavy vehicle, 1: empty vehicle)

4 : Top light status (0: operation, 1: to be transported, 2: electrical adjustment, 3: suspension, 4: help, 5: suspension)

5 : Business status (0: surface road, 1: fast road)

6 : Service status (0: no brake, 1: brake)

7 : Nonsense field

8 : Receiving date

9 : GPS time

10 :longitude

11 : latitude

12 : speed

13 : The direction of

14 : Number of satellites

15 : Nonsense field

In [5]:
## rename columns to their original name --> in VEHICULAR DATASET
data_0 = data_0.rename(columns={0 : "Car Number",
                       2 : "Business Status",
                       9 : "GPS time", 
                       10: "longitude", 
                       11: "latitude",
                       12: "Speed"})

data_0.columns.values

array(['Car Number', 1, 'Business Status', 3, 4, 5, 6, 7, 8, 'GPS time',
       'longitude', 'latitude', 'Speed', 13, 14, 15], dtype=object)

In [18]:
## take only columns --> ["Car Number", "Business Status", "GPS time", "longitude", "latitude", "Speed"] into account
vehicleData_0 = data_0[["Car Number", "Business Status", "GPS time",
                        "longitude", "latitude", "Speed"]]

vehicleData_0.head()                

,Car Number,Business Status,GPS time,longitude,latitude,Speed
0,18852,0,2018-03-31 23:59:48,121.498510,31.289317,68.0
1,27772,0,2018-03-31 23:59:59,121.560292,31.244518,47.0
2,10373,0,2018-04-01 00:00:02,121.412063,31.278722,0.0
3,10708,0,2018-04-01 00:00:02,121.666038,31.165347,0.0
4,17988,0,2018-04-01 00:00:01,121.460905,31.342570,34.0


In [19]:
vehicleData_0.shape

(59955, 6)

save the vehicle dataset_0 to .csv file

In [20]:
vehicleData_0.to_csv("vehicleData_0.csv", index=False)

install haversine package, we will use haversine as distance metric to calculate distance between two coordinates(lat, long)

In [14]:
pip install haversine

In [16]:
from haversine import haversine, Unit

In [23]:
point1 = vehicleData_0["latitude"][0], vehicleData_0["longitude"][0]    ## lat, long

point2 = tower_locations["latitude"][0], tower_locations["longitude"][0]   ## lat, long

havesine(point1, point2) will return distance between the 2 coordinates(lat, long) in km

In [25]:
haversine(point1, point2)

1212.1598649329817

testing it on first 1000 data points

In [30]:
df = pd.read_csv("vehicleData_0.csv", nrows=1000)

In [31]:
df.shape

(1000, 6)

In [32]:
df.head()

,Car Number,Business Status,GPS time,longitude,latitude,Speed
0,18852,0,2018-03-31 23:59:48,121.498510,31.289317,68.0
1,27772,0,2018-03-31 23:59:59,121.560292,31.244518,47.0
2,10373,0,2018-04-01 00:00:02,121.412063,31.278722,0.0
3,10708,0,2018-04-01 00:00:02,121.666038,31.165347,0.0
4,17988,0,2018-04-01 00:00:01,121.460905,31.342570,34.0


In [29]:
tower_locations.shape

(2769, 2)

1. take one row from df. 
2. for each coordinate(latitude, longitude) in df, calculates distance from one cell_tower coordinates in tower_locations dataframe.
3. Do step 2 for each row in tower_locations dataFrame
4. repeat 1,2,3 for each data point in df

I have taken one data point from df dataFrame. All below calculations are for one single data point, p1(say) in df. We calculate HAVERSINE distance of all cell towers from this point. 
After the distance calculation, we will only take those cell tower locations which are in 2 km range to point p1.

In [39]:
point_1 = df["latitude"][1], df["longitude"][1]

distance = {} ## this dictionary will contain each tower_location as keys and haversine distance as value

for i in range(len(list(tower_locations["latitude"]))):
  point_i = tower_locations["latitude"][i], tower_locations["longitude"][i]

  distance[point_i] = haversine(point1=point_1, point2=point_i)


In [40]:
distance

{(22.522802352905273, 114.2187957763672): 1211.90786159734,
 (24.28481292724609, 102.9990692138672): 1979.9853297704394,
 (25.222206115722656, 117.08631896972656): 800.1160052244585,
 (26.13932991027832, 103.07855987548828): 1887.1932801788212,
 (26.21511459350586, 109.74465942382812): 1279.6015160702716,
 (28.73874282836914, 120.64060974121094): 292.36424532458153,
 (28.81262969970703, 115.95295715332031): 603.6402898550425,
 (29.15177917480469, 120.98556518554688): 239.1661656568626,
 (29.263843536376953, 115.02315521240234): 665.2578018975846,
 (29.52626609802246, 119.91048431396484): 248.08038437858923,
 (30.442176818847656, 120.61872863769531): 126.64706005949883,
 (30.6799430847168, 104.06792449951172): 1667.348711049169,
 (30.70576286315918, 121.33512115478516): 63.63705609214684,
 (30.71552658081055, 121.35107421875): 62.11077128703796,
 (30.71604347229004, 121.2910385131836): 64.12524968903794,
 (30.7161865234375, 121.34949493408205): 62.08982235328574,
 (30.718107223510746, 1

In [41]:
## sorting distance dictionary based on distance between point p1 and cell tower locations
distance = sorted(distance.items(), key=lambda item: item[1])

distance

[((31.243974685668945, 121.55575561523438), 0.43547505587311974),
 ((31.23881530761719, 121.5593490600586), 0.6404166865178912),
 ((31.242156982421875, 121.56878662109376), 0.849174513189156),
 ((31.252105712890625, 121.56398010253906), 0.9136634438627014),
 ((31.24040412902832, 121.56918334960938), 0.9611348663574579),
 ((31.2390251159668, 121.55162048339844), 1.0260094408400562),
 ((31.24486541748047, 121.54950714111328), 1.0260153441227875),
 ((31.24641036987305, 121.5485382080078), 1.1370328396682599),
 ((31.25474548339844, 121.55608367919922), 1.2055581435995228),
 ((31.235631942749023, 121.5678482055664), 1.2216335139217354),
 ((31.25615119934082, 121.55784606933594), 1.3142854316280144),
 ((31.23365974426269, 121.56580352783205), 1.316187999179459),
 ((31.252241134643555, 121.57093811035156), 1.327311137086762),
 ((31.233009338378906, 121.55638885498048), 1.3324236786781074),
 ((31.252485275268555, 121.54961395263672), 1.3473197117877065),
 ((31.23590850830078, 121.5706710815429

In [43]:
type(distance)

list

In [45]:
distance[0][1]

0.43547505587311974

creating a dictionary which will contain only nearest cell towers in the range of 2 km, 
key : location(lat, long) || value : haversine_distance

In [46]:
nearest_cellTowers = {}

for k in distance:
  if k[1] < float(2):
    nearest_cellTowers[k[0]] = k[1]

In [47]:
nearest_cellTowers

{(31.22699737548828, 121.5563735961914): 1.983507773071756,
 (31.2282829284668, 121.55667877197266): 1.8376552742170664,
 (31.230287551879886, 121.55711364746094): 1.610951022721171,
 (31.230670928955078, 121.55284881591795): 1.6945603956692266,
 (31.231035232543945, 121.55560302734376): 1.5640943634040934,
 (31.2317008972168, 121.5465087890625): 1.9360795104860633,
 (31.23239517211914, 121.5646743774414): 1.4109205344093343,
 (31.232728958129886, 121.56693267822266): 1.4549986208403816,
 (31.233009338378906, 121.55638885498048): 1.3324236786781074,
 (31.23365974426269, 121.56580352783205): 1.316187999179459,
 (31.235631942749023, 121.5678482055664): 1.2216335139217354,
 (31.23590850830078, 121.57067108154295): 1.3748377793545563,
 (31.237022399902344, 121.54618072509766): 1.5794010316930114,
 (31.23786926269531, 121.54320526123048): 1.784773231250123,
 (31.23881530761719, 121.5593490600586): 0.6404166865178912,
 (31.2390251159668, 121.55162048339844): 1.0260094408400562,
 (31.23976325

The number of cell towers in the range of 2 km of a data point

In [48]:
len(nearest_cellTowers)

34

There are 34 cell towers in the radius of 2 km to df["latitude"][1], df["longitude"][1]

#### this calculation is yet to be streached for each point in df. 
to do this, I need --
1. how to store the calculations, which data structure